In [17]:
import os
from dotenv import load_dotenv
from openai import OpenAI


In [18]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-;")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [19]:
openai = OpenAI()


In [20]:
# STEP 1 - Define the function GPT can use
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [21]:
# STEP 2 - Let GPT decide to call the function
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What's the weather like in Mumbai today?"}
    ],
    tools=[{"type": "function", "function": functions[0]}],
    tool_choice="auto"
)

print(response.choices[0].message.content)

None


In [22]:
#STEP 3 - .Extract what it wants to call
import json

# Step 2: Extract tool call
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

In [23]:
# STEP 4 - Run the real function in your backend / Simulate function
def get_current_weather(location, unit='celsius'):
    # API 
    return f"The weather in {location} is 31° {unit} with scattered clouds."

# Call your function manually
result = get_current_weather(**arguments)

In [29]:
# Step 5: Give the result back so GPT can finish the reply
follow_up = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What' the weather like in Mumbai today?"},
        response.choices[0].message,  # tool_call message
        {"role": "tool", "tool_call_id": tool_call.id, "content": result}
    ]
)

print(follow_up.choices[0].message.content)

The weather in Mumbai today is 31° Celsius with scattered clouds.
